In [1]:
%cd ../../

/home/PJWSTK/s14028/Engine


In [2]:
import keras
import scipy.io as mat

from common import *
from augmentation import add_pmap
from testing import test_model
from testing import best_cop_diff
from testing import mse
from testing import mae

from CNN.CNN_v2_2 import CNN_v2_2

Using TensorFlow backend.


In [3]:
perspective = mat.loadmat("mall_dataset/perspective_roi.mat")["pMapN"]

perspective /= np.min(perspective)
perspective = np.round(perspective).astype(np.uint8)

train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

image_train = add_pmap(image_train, perspective)
image_test = add_pmap(image_test, perspective)

In [4]:
cnn = CNN_v2_2((480, 640, 4), split_into_parts=40)

images = cnn._prepare_images(image_test)
anwsers = cnn._prepare_anwsers(person_coo_test)

In [5]:
cop = np.linspace(0, 1, 11)[1:-1]

In [6]:
prefix = "CNN/CNN_v2/weights/"

In [7]:
model_names = ["cnn_v2_2_40_1e_3_20",
               "cnn_v2_2_40_1e_3_30",
               "cnn_v2_2_40_1e_5_01_0.83",
               "cnn_v2_2_40_1e_5_03_0.86",
               "cnn_v2_2_40_1e_3_50",
               "cnn_v2_2_40_1e_3_60",
               "cnn_v2_2_40_1e_3_90",
               "cnn_v2_2_40_1e_3_100",
               "cnn_v2_2_40_1e_5_next_01_0.84",
               "cnn_v2_2_40_1e_5_next_06_0.84"]

model_names = [prefix + name for name in model_names]

In [8]:
models = [CNN_v2_2((480, 640, 4), split_into_parts=40) for i in range(len(model_names))]

In [9]:
results = {}

for model_name, model in zip(model_names, models):
  model.def_model()
  model.model.fit(images[:1], anwsers[:1])
  
  model.model = keras.utils.multi_gpu_model(model.model, gpus=2, cpu_merge=False)
  model.model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  
  model.model.load_weights(model_name)
  
  results[model_name] = test_model(model=model, images=image_test, counts=count_test, cut_of_points=cop)

Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 0.7004 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 0.6995 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 0.6998 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 0.7001 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 0.6998 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 5s 5s/step - loss: 0.6996 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 4s 4s/step - loss: 0.6995 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 5s 5s/step - loss: 0.6999 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 5s 5s/step - loss: 0.7002 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 6s 6s/step - loss: 0.6996 - acc: 1.0000


In [10]:
for name, result in results.items():
  diff = best_cop_diff(result, count_test)
  
  print(f"Model {name} behaved:")
  
  print(f"For cut-of-point {diff[0]} had accuracy {diff[1]}:")
  print(diff[2])
  
  print(f"With MSE {mse(diff[2])}")
  print(f"With MAE {mae(diff[2])}", end="\n\n")

Model CNN/CNN_v2/weights/cnn_v2_2_40_1e_3_20 behaved:
For cut-of-point 0.1 had accuracy 0.0:
[1546 1552 1555 1550 1544 1557 1557 1568 1571 1576 1576 1578 1570 1572
 1577 1567 1563 1571 1566 1564 1565 1564 1557 1563 1560 1560 1569 1566
 1569 1569 1566 1560 1559 1564 1563 1558 1566 1569 1559 1565 1564 1562
 1560 1568 1566 1561 1560 1554 1560 1565 1562 1569 1571 1573 1570 1565
 1548 1554 1546 1539 1542 1537 1538 1548 1553 1553 1546 1545 1537 1554
 1559 1556 1556 1558 1559 1562 1564 1564 1557 1564 1559 1557 1560 1562
 1565 1559 1562 1564 1562 1563 1560 1558 1559 1542 1545 1545 1547 1547
 1553 1543 1555 1553 1557 1558 1551 1557 1552 1551 1537 1543 1548 1553
 1545 1540 1542 1547 1556 1559 1557 1558 1547 1551 1553 1552 1556 1556
 1559 1562 1552 1555 1556 1552 1551 1547 1538 1557 1556 1563 1553 1555
 1562 1554 1551 1545 1541 1555 1554 1560 1556 1563 1563 1564 1560 1554
 1552 1549 1550 1546 1560 1562 1559 1557 1560 1568 1562 1550 1559 1561
 1562 1561 1554 1558 1565 1563 1564 1560 1559 1560 1559